In [1]:
pip install git+https://github.com/JustAnotherArchivist/snscrape.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/JustAnotherArchivist/snscrape.git to /tmp/pip-req-build-ky_9_3fl
  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git /tmp/pip-req-build-ky_9_3fl
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit 614d4c2029a62d348ca56598f87c425966aaec66
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for snscrape: filename=snscrape-0.6.2.20230321.dev53+g614d4c2-py3-none-any.whl size=75022 sha256=896a51075de870e94fe96705ff38ca590334eb7258ead7774f95f5d29b02985d
  Stored in directory: /tmp/pip-ephem-wheel-cache-36495wq9/wheels/05/e9/f7/57056e7c7e44b1feed932fa49fdec9d706c4f563e37160ab74
Successfully built snscrape


In [2]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from datetime import datetime, timedelta

In [3]:
hashtag = "#yapayzeka"

In [4]:
today = datetime.now().date()
month_ago = today - timedelta(days=150)

In [5]:
query = f'{hashtag} since:{month_ago}'

In [6]:
tweets_list = []
max_tweets = 10000
for i, tweet in enumerate(sntwitter.TwitterHashtagScraper(query).get_items()):
    if i >= max_tweets:
        break
    tweets_list.append([tweet.date, tweet.id,tweet.username,tweet.user.id,tweet.hashtags, tweet.content, tweet.likeCount, tweet.retweetCount, tweet.replyCount])

<ipython-input-6-ef63abacc4f0>:6: DeprecatedFeatureWarning: username is deprecated, use user.username instead
  tweets_list.append([tweet.date, tweet.id,tweet.username,tweet.user.id,tweet.hashtags, tweet.content, tweet.likeCount, tweet.retweetCount, tweet.replyCount])
<ipython-input-6-ef63abacc4f0>:6: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets_list.append([tweet.date, tweet.id,tweet.username,tweet.user.id,tweet.hashtags, tweet.content, tweet.likeCount, tweet.retweetCount, tweet.replyCount])


In [7]:
df = pd.DataFrame(tweets_list, columns=['Datetime', 'Tweet Id', 'username' ,'userid','hastags' ,  'Content', 'Like Count', 'Retweet Count', 'Reply Count'])

In [8]:
df['Datetime'] = pd.to_datetime(df['Datetime']).dt.tz_localize(None)

In [11]:
df.to_excel('Yapayzeka.xlsx', index=False)

In [14]:
df = pd.read_excel('Yapayzeka.xlsx')

In [15]:
source_list = []
target_list = []
type_list = []
weight_list = []

In [16]:
for index, row in df.iterrows():
    hashtags = row['hastags']
    username = row['username']
    like_count = row['Like Count']
    retweet_count = row['Retweet Count']
    reply_count = row['Reply Count']

    hashtag_list = hashtags.strip("[]").replace("'", "").split(',')

    for hashtag in hashtag_list:
        hashtag = hashtag.strip()

        if hashtag.lower() == 'yapayzeka':
            source = 'Yapayzeka'
            target = username
            relation_type = 'mention'
            weight = 1+like_count + retweet_count + reply_count
            source_list.append(source)
            target_list.append(target)
            type_list.append(relation_type)
            weight_list.append(weight)

In [17]:
data = {
    'Source': source_list,
    'Target': target_list,
    'Type': type_list,
    'Weight': weight_list
}

In [18]:
df_new = pd.DataFrame(data)

In [19]:
df_new.to_csv('Yapayzeka_gephi', index=False)